In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from matplotlib import gridspec
import ast
import os

#os.chdir('/home/bojk/Data/') #this is when opened in '/home/bojk/Data/notebooks' on linux Quakelab
os.chdir('/Users/bojk/Google Drive/QuakeLab/Data/')
import sys
sys.path.append('/home/bojk/Data/minimeta_pyfiles/')
sys.path.append('/Users/bojk/Google Drive/QuakeLab/minimeta_pyfiles/')
import imp
import HTSeq
from scipy.stats import powerlaw
import minimetagenome_analysis as minimeta

import mock_metagenome_creation as mock
import minimetagenome_dimensionality as miniD
imp.reload(mock)
#import reassembly as rea
#import subcluster_bins as sub
#import checkm_out_plot as cop
#import checkm_out as co

import matplotlib as mpl
mpl.rcParams['axes.labelsize'] = 15
mpl.rcParams['xtick.labelsize'] = 15
mpl.rcParams['ytick.labelsize'] = 15
mpl.rcParams['axes.titlesize'] = 15
mpl.rcParams['figure.titlesize'] = 18
mpl.rcParams['legend.fontsize'] = 14

from sklearn.manifold import TSNE
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

# SagMAT KMERS AND MINIMETA CHIP STUFF

In [3]:
#metagenome_fasta = '/home/datastorage/ASSEMBLY_DATA/PermafrostSagMAT/Combined_Analysis/super_contigs.PermafrostSagMAT.fasta'        
metagenome_fasta = 'Permafrost/SagMAT/super_contigs.PermafrostSagMAT.fasta'
save_dir = 'Permafrost/SagMAT/pickles/'

In [4]:
maindf = pd.read_pickle('Permafrost/SagMAT/pickles/SagMAT_maindf')
maindf.head(2)

,Scaffold ID,Scaffold Name,Genome ID,Genome,Gene Count,Sequence Length,GC Content,Read Depth,Lineage Domain,Lineage Phylum,...,y_40_a,y_40_n,y_50_a,y_50_n,y_60_a,y_60_n,y_70_a,y_70_n,y_80_a,y_80_n
SuperContig_k91_275_flag=0_multi=8.2373_len=3205_SegMat_25cm_Bulk,3300022816 assembled Ga0255657_100004,100004,3300022816,Permafrost soil microbial communities from Sag...,4,3205,0.58,1,Bacteria,Unassigned,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
SuperContig_k91_360_flag=1_multi=6.0000_len=3358_SegMat_25cm_Bulk,3300022816 assembled Ga0255657_100006,100006,3300022816,Permafrost soil microbial communities from Sag...,4,3358,0.68,1,Bacteria,Unassigned,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
maindf.shape

(65990, 42)

In [6]:
contigdf = maindf[['Sequence Length','GC Content','Read Depth']].copy()
contigdf.rename(index=str,columns={'Sequence Length':'Sequence length','GC Content':'GC'},inplace=True)

In [7]:
chipAbs = pd.read_pickle('Permafrost/SagMAT/pickles/SagMAT_chipdf_bpcounts')
chipMini = chipAbs[~chipAbs.index.str.contains('Bulk')].copy()
chipMini.drop(['SagMAT_25cm_Bulk','SagMAT_15cm_Bulk'],axis=1,inplace=True)
chipMini.shape

(33580, 365)

In [16]:
idx = chipMini[chipMini>0
              ].dropna(how='all').fillna(0).index
print(contigdf.loc[idx].shape)
print(contigdf.loc[idx,'Sequence length'].sum())
print(contigdf.loc[idx,'Sequence length'].median())

(15319, 3)
110750347
5138.0


In [8]:
chipMini[chipMini>0].dropna(how='all').fillna(0).shape

(15319, 365)

In [9]:
chimMinifinal = chipMini[chipMini>0].dropna(how='all').fillna(0).copy()

In [57]:

imp.reload(miniD)
maindir = 'Permafrost/SagMAT/chipPCA/'
#SMmini_alltsne = pd.DataFrame('Permafrost/SagMAT/pickles/SMminiCovOnly6mers_all_tSNEs')
SMmini_alltsne = miniD.perform_complete_analysis_Coverage(chimMinifinal,6,contigdf,maindir,'SMminiCovOnly')

building tSNE of all 365 dimensions
finished building main tSNE, this took 526.74 seconds
performing cluster sweep of tSNE of all 365 dimensions
Performing PCA...
building tSNE of 4 PCs
finished building tSNE of 4 PCs, this took 396.40 seconds
Index([0, 1, 'Sequence length', 'GC'], dtype='object')
building tSNE of 6 PCs
finished building tSNE of 6 PCs, this took 408.98 seconds
Index([0, 1, 'Sequence length', 'GC'], dtype='object')
building tSNE of 9 PCs
finished building tSNE of 9 PCs, this took 385.24 seconds
Index([0, 1, 'Sequence length', 'GC'], dtype='object')
building tSNE of 13 PCs
finished building tSNE of 13 PCs, this took 384.55 seconds
Index([0, 1, 'Sequence length', 'GC'], dtype='object')
building tSNE of 18 PCs
finished building tSNE of 18 PCs, this took 391.09 seconds
Index([0, 1, 'Sequence length', 'GC'], dtype='object')
building tSNE of 26 PCs
finished building tSNE of 26 PCs, this took 443.27 seconds
Index([0, 1, 'Sequence length', 'GC'], dtype='object')
building tSNE o

In [18]:
maindir = 'Permafrost/SagMAT/chipPCA/'
SMmini_alltsne3kb = miniD.perform_complete_analysis_Coverage(chimMinifinal[chimMinifinal>3000].dropna(how='all').fillna(0),6,contigdf,maindir,'SMminiCovOnly3kbThres')

building tSNE of all 365 dimensions
finished building main tSNE, this took 394.43 seconds
performing cluster sweep of tSNE of all 365 dimensions
Performing PCA...
building tSNE of 4 PCs
finished building tSNE of 4 PCs, this took 312.12 seconds
Index([0, 1, 'Sequence length', 'GC'], dtype='object')
building tSNE of 6 PCs
finished building tSNE of 6 PCs, this took 334.73 seconds
Index([0, 1, 'Sequence length', 'GC'], dtype='object')
building tSNE of 9 PCs
finished building tSNE of 9 PCs, this took 299.72 seconds
Index([0, 1, 'Sequence length', 'GC'], dtype='object')
building tSNE of 13 PCs
finished building tSNE of 13 PCs, this took 297.62 seconds
Index([0, 1, 'Sequence length', 'GC'], dtype='object')
building tSNE of 18 PCs
finished building tSNE of 18 PCs, this took 296.52 seconds
Index([0, 1, 'Sequence length', 'GC'], dtype='object')
building tSNE of 26 PCs
finished building tSNE of 26 PCs, this took 298.45 seconds
Index([0, 1, 'Sequence length', 'GC'], dtype='object')
building tSNE o

In [19]:
SMmini_alltsne5kb = miniD.perform_complete_analysis_Coverage(chimMinifinal[chimMinifinal>5000].dropna(how='all').fillna(0),6,contigdf,maindir,'SMminiCovOnly5kbThres')

building tSNE of all 365 dimensions
finished building main tSNE, this took 165.61 seconds
performing cluster sweep of tSNE of all 365 dimensions
Performing PCA...
building tSNE of 4 PCs
finished building tSNE of 4 PCs, this took 136.71 seconds
Index([0, 1, 'Sequence length', 'GC'], dtype='object')
building tSNE of 6 PCs
finished building tSNE of 6 PCs, this took 136.93 seconds
Index([0, 1, 'Sequence length', 'GC'], dtype='object')
building tSNE of 9 PCs
finished building tSNE of 9 PCs, this took 135.74 seconds
Index([0, 1, 'Sequence length', 'GC'], dtype='object')
building tSNE of 13 PCs
finished building tSNE of 13 PCs, this took 143.23 seconds
Index([0, 1, 'Sequence length', 'GC'], dtype='object')
building tSNE of 18 PCs
finished building tSNE of 18 PCs, this took 136.38 seconds
Index([0, 1, 'Sequence length', 'GC'], dtype='object')
building tSNE of 26 PCs
finished building tSNE of 26 PCs, this took 137.20 seconds
Index([0, 1, 'Sequence length', 'GC'], dtype='object')
building tSNE o

In [20]:
SMmini_alltsne7kb = miniD.perform_complete_analysis_Coverage(chimMinifinal[chimMinifinal>7000].dropna(how='all').fillna(0),6,contigdf,maindir,'SMminiCovOnly7kbThres')

building tSNE of all 365 dimensions
finished building main tSNE, this took 118.72 seconds
performing cluster sweep of tSNE of all 365 dimensions
Performing PCA...
building tSNE of 4 PCs
finished building tSNE of 4 PCs, this took 81.75 seconds
Index([0, 1, 'Sequence length', 'GC'], dtype='object')
building tSNE of 6 PCs
finished building tSNE of 6 PCs, this took 80.88 seconds
Index([0, 1, 'Sequence length', 'GC'], dtype='object')
building tSNE of 9 PCs
finished building tSNE of 9 PCs, this took 80.57 seconds
Index([0, 1, 'Sequence length', 'GC'], dtype='object')
building tSNE of 13 PCs
finished building tSNE of 13 PCs, this took 86.33 seconds
Index([0, 1, 'Sequence length', 'GC'], dtype='object')
building tSNE of 18 PCs
finished building tSNE of 18 PCs, this took 83.18 seconds
Index([0, 1, 'Sequence length', 'GC'], dtype='object')
building tSNE of 26 PCs
finished building tSNE of 26 PCs, this took 83.45 seconds
Index([0, 1, 'Sequence length', 'GC'], dtype='object')
building tSNE of 38 P

In [21]:
SMmini_alltsne10kb = miniD.perform_complete_analysis_Coverage(chimMinifinal[chimMinifinal>10000].dropna(how='all').fillna(0),6,contigdf,maindir,'SMminiCovOnly10kbThres')

building tSNE of all 365 dimensions
finished building main tSNE, this took 50.99 seconds
performing cluster sweep of tSNE of all 365 dimensions
Performing PCA...
building tSNE of 4 PCs
finished building tSNE of 4 PCs, this took 38.13 seconds
Index([0, 1, 'Sequence length', 'GC'], dtype='object')
building tSNE of 6 PCs
finished building tSNE of 6 PCs, this took 37.17 seconds
Index([0, 1, 'Sequence length', 'GC'], dtype='object')
building tSNE of 9 PCs
finished building tSNE of 9 PCs, this took 39.29 seconds
Index([0, 1, 'Sequence length', 'GC'], dtype='object')
building tSNE of 13 PCs
finished building tSNE of 13 PCs, this took 40.53 seconds
Index([0, 1, 'Sequence length', 'GC'], dtype='object')
building tSNE of 18 PCs
finished building tSNE of 18 PCs, this took 40.60 seconds
Index([0, 1, 'Sequence length', 'GC'], dtype='object')
building tSNE of 26 PCs
finished building tSNE of 26 PCs, this took 41.95 seconds
Index([0, 1, 'Sequence length', 'GC'], dtype='object')
building tSNE of 38 PC

In [56]:
idx = SMmini_alltsne.index
SMmaindf_mini = maindf.loc[idx].copy()
SMmaindf_mini = SMmaindf_mini.join(SMmini_alltsne)

NameError: name 'SMmini_alltsne' is not defined

In [24]:
"""SMmini_alltsne
#[['x_PC13','y_PC13']])
"""
idx = SMmini_alltsne3kb.index
SMmaindf_mini3kb = maindf.loc[idx].copy()
SMmaindf_mini3kb = SMmaindf_mini3kb.join(SMmini_alltsne3kb)#[['x_PC13','y_PC13']])

idx = SMmini_alltsne5kb.index
SMmaindf_mini5kb = maindf.loc[idx].copy()
SMmaindf_mini5kb = SMmaindf_mini5kb.join(SMmini_alltsne5kb)#[['x_PC13','y_PC13']])

idx = SMmini_alltsne7kb.index
SMmaindf_mini7kb = maindf.loc[idx].copy()
SMmaindf_mini7kb = SMmaindf_mini7kb.join(SMmini_alltsne7kb)#[['x_PC13','y_PC13']])

idx = SMmini_alltsne10kb.index
SMmaindf_mini10kb = maindf.loc[idx].copy()
SMmaindf_mini10kb = SMmaindf_mini10kb.join(SMmini_alltsne10kb)#[['x_PC13','y_PC13']])



In [102]:
imp.reload(miniD)
#main = pd.read_csv('Permafrost/SIMR/WestDock_metadata.table').set_index('Unnamed: 0')
#tsnedf_WDminiCov = pd.read_pickle(maindir+'WDminiCovOnly6mers_all_tSNEs')
miniD.plotPCs(tsnedf_WDminiCov,main,maindir,'WDminiChipAbsCov') 

In [ ]:
minCS = 100
keys = ['min_cluster_size','min_samples','cluster_selection_method','allow_single_cluster'
             ,'perplexity_na','expt_name','write_fasta','write_df','is_this_a_series','outdir']
values = [minCS,1,'leaf',False,'PC13','WestDock_miniPC13',fasta,write,'NO','Permafrost/SagMAT/']
kwargs = dict(zip(keys,values))
for pcs in [9,13,18,26,38,55,78,113]:
    pc = str(pcs)
    kwargs['perplexity_na'] = 'PC'+pc
    kwargs['expt_name'] = 'SagMAT_miniPC'+pc+'_0kbthres_minCS'+str(minCS)
    maindf_mini = minimeta.cluster_main_tsne(WDmaindf_mini3,**kwargs)

In [55]:
SMmaindf_mini3kb.head(2)

,Scaffold ID,Scaffold Name,Genome ID,Genome,Gene Count,Sequence Length,GC Content,Read Depth,Lineage Domain,Lineage Phylum,...,x_PC113,y_PC113,x_PC162,y_PC162,x_PC234,y_PC234,x_PC336,y_PC336,DBclusternum,Bin
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
SuperContig_NODE_3598_length_7534_cov_2.88191,3300022816 assembled Ga0255657_157610,157610,3300022816,Permafrost soil microbial communities from Sag...,13,7534,0.38,1,Unassigned,Unassigned,...,26.000343,-13.918259,-23.339903,-27.095160,28.827444,38.500633,52.13921,14.260105,-1,-01
SuperContig_NODE_3222_length_8048_cov_4.20292,3300022816 assembled Ga0255657_157234,157234,3300022816,Permafrost soil microbial communities from Sag...,2,8048,0.44,1,Unassigned,Unassigned,...,57.267860,7.114372,-9.631462,42.196144,15.303872,-64.097580,39.74527,55.419998,48,048


In [51]:
imp.reload(minimeta)
fasta,write = 'YES','YES'
minCS = 70
keys = ['min_cluster_size','min_samples','cluster_selection_method','allow_single_cluster'
             ,'perplexity_na','expt_name','write_fasta','write_df','is_this_a_series','outdir']
values = [minCS,1,'leaf',False,'PC13','WestDock_miniPC13',fasta,write,'NO','Permafrost/SagMAT/']
kwargs = dict(zip(keys,values))
for pcs in [9,13,18,26,38,55,78,113]:
    pc = str(pcs)
    kwargs['perplexity_na'] = 'PC'+pc
    kwargs['expt_name'] = 'SagMAT_miniPC'+pc+'_3kbthres_minCS'+str(minCS)
    maindf_mini = minimeta.cluster_main_tsne(SMmaindf_mini3kb,**kwargs) # cluster whatever pcs you want here.



In [52]:

minCS = 40
keys = ['min_cluster_size','min_samples','cluster_selection_method','allow_single_cluster'
             ,'perplexity_na','expt_name','write_fasta','write_df','is_this_a_series','outdir']
values = [minCS,1,'eom',False,'PC13','WestDock_miniPC13',fasta,write,'NO','Permafrost/SagMAT/']
kwargs = dict(zip(keys,values))
for pcs in [9,13,18,26,38,55,78,113]:
    pc = str(pcs)
    kwargs['perplexity_na'] = 'PC'+pc
    kwargs['expt_name'] = 'SagMAT_miniPC'+pc+'_5kbthres_minCS'+str(minCS)
    maindf_mini = minimeta.cluster_main_tsne(SMmaindf_mini5kb,**kwargs) # cluster whatever pcs you want here.



In [53]:

minCS = 30
keys = ['min_cluster_size','min_samples','cluster_selection_method','allow_single_cluster'
             ,'perplexity_na','expt_name','write_fasta','write_df','is_this_a_series','outdir']
values = [minCS,1,'leaf',False,'PC13','WestDock_miniPC13',fasta,write,'NO','Permafrost/SagMAT/']
kwargs = dict(zip(keys,values))
for pcs in [9,13,18,26,38,55,78,113]:
    pc = str(pcs)
    kwargs['perplexity_na'] = 'PC'+pc
    kwargs['expt_name'] = 'SagMAT_miniPC'+pc+'_7kbthres_minCS'+str(minCS)
    maindf_mini = minimeta.cluster_main_tsne(SMmaindf_mini7kb,**kwargs) # cluster whatever pcs you want here.



In [54]:

minCS = 25
keys = ['min_cluster_size','min_samples','cluster_selection_method','allow_single_cluster'
             ,'perplexity_na','expt_name','write_fasta','write_df','is_this_a_series','outdir']
values = [minCS,1,'leaf',False,'PC13','WestDock_miniPC13',fasta,write,'NO','Permafrost/SagMAT/']
kwargs = dict(zip(keys,values))
for pcs in [9,13,18,26,38,55,78,113]:
    pc = str(pcs)
    kwargs['perplexity_na'] = 'PC'+pc
    kwargs['expt_name'] = 'SagMAT_miniPC'+pc+'_10kbthres_minCS'+str(minCS)
    maindf_mini = minimeta.cluster_main_tsne(SMmaindf_mini10kb,**kwargs) # cluster whatever pcs you want here.



In [99]:
#get tsne cols in the right format (write function to do this, then pass on to 
#                                  standard minimeta pipeline)
imp.reload(minimeta)
"""
minCS ['min_cluster_size']
minS ['min_samples']
CSM ['cluster_selection_method']
ASC ['allow_single_cluster']
perp ['perplexity_na'] # this is a string of the form '40_a' or '60_n' (so perplexity and absolute or normalized tsne)
expt_name ['expt_name']
write_fasta ['write_fasta'] # save the fasta Y/n (as in: do dry run clustering first, as fasta writing takes time)
writedf ['write_df']
series ['is_this_a_series']
"""
 # cluster whatever pcs you want here.










In [92]:
maindf_mini.groupby('DBclusternum').sum()[['Sequence Length','Read Depth']]
maindf_mini.shape

(16619, 51)

In [97]:
size = maindf_mini['Sequence Length']
maindf_mini.plot.scatter('x_PC13','y_PC13',s=size.divide(3e2).astype(float),alpha=.05)#,ax=ax1,c=colors)

In [100]:
# compute WD kmers
fastapath = 'Permafrost/WestDock/super_contigs.PermafrostWestDock.fasta'
savedir = 'Permafrost/WestDock/pickles/'
contig6mers,k6mers  = mock.make_kmertable_from_fasta_contigs(fastapath,6,savedir)

Total number of fasta sequences:45613
----------------------------------------------------
Started collecting kmers, dimensionality = 4096
Local time: 22:24
----------------------------------------------------
----------------------------------------------------
Progress: 1.1 %, local time: 22:37
Total elapsed time is 12.73 minutes
Saving current contig kmers to pickle...
relative path = Permafrost/WestDock/pickles/kmerdf_from_metagenome_fasta_6mer_incomplete.pickle
----------------------------------------------------
----------------------------------------------------
Progress: 2.19 %, local time: 22:50
Total elapsed time is 25.73 minutes
Saving current contig kmers to pickle...
relative path = Permafrost/WestDock/pickles/kmerdf_from_metagenome_fasta_6mer_incomplete.pickle
----------------------------------------------------
----------------------------------------------------
Progress: 3.29 %, local time: 23:3
Total elapsed time is 38.74 minutes
Saving current contig kmers to pickle

----------------------------------------------------
Progress: 24.12 %, local time: 3:23
Total elapsed time is 298.78 minutes
Saving current contig kmers to pickle...
relative path = Permafrost/WestDock/pickles/kmerdf_from_metagenome_fasta_6mer_incomplete.pickle
----------------------------------------------------
----------------------------------------------------
Progress: 25.21 %, local time: 3:36
Total elapsed time is 312.24 minutes
Saving current contig kmers to pickle...
relative path = Permafrost/WestDock/pickles/kmerdf_from_metagenome_fasta_6mer_incomplete.pickle
----------------------------------------------------
----------------------------------------------------
Progress: 26.31 %, local time: 3:50
Total elapsed time is 325.86 minutes
Saving current contig kmers to pickle...
relative path = Permafrost/WestDock/pickles/kmerdf_from_metagenome_fasta_6mer_incomplete.pickle
----------------------------------------------------
----------------------------------------------------

relative path = Permafrost/WestDock/pickles/kmerdf_from_metagenome_fasta_6mer_incomplete.pickle
----------------------------------------------------
----------------------------------------------------
Progress: 48.23 %, local time: 9:19
Total elapsed time is 654.76 minutes
Saving current contig kmers to pickle...
relative path = Permafrost/WestDock/pickles/kmerdf_from_metagenome_fasta_6mer_incomplete.pickle
----------------------------------------------------
----------------------------------------------------
Progress: 49.33 %, local time: 9:35
Total elapsed time is 671.25 minutes
Saving current contig kmers to pickle...
relative path = Permafrost/WestDock/pickles/kmerdf_from_metagenome_fasta_6mer_incomplete.pickle
----------------------------------------------------
----------------------------------------------------
Progress: 50.42 %, local time: 9:52
Total elapsed time is 687.99 minutes
Saving current contig kmers to pickle...
relative path = Permafrost/WestDock/pickles/kmerdf_f

----------------------------------------------------
Progress: 71.25 %, local time: 16:36
Total elapsed time is 1092.44 minutes
Saving current contig kmers to pickle...
relative path = Permafrost/WestDock/pickles/kmerdf_from_metagenome_fasta_6mer_incomplete.pickle
----------------------------------------------------
----------------------------------------------------
Progress: 72.35 %, local time: 16:56
Total elapsed time is 1112.51 minutes
Saving current contig kmers to pickle...
relative path = Permafrost/WestDock/pickles/kmerdf_from_metagenome_fasta_6mer_incomplete.pickle
----------------------------------------------------
----------------------------------------------------
Progress: 73.44 %, local time: 17:15
Total elapsed time is 1131.52 minutes
Saving current contig kmers to pickle...
relative path = Permafrost/WestDock/pickles/kmerdf_from_metagenome_fasta_6mer_incomplete.pickle
----------------------------------------------------
----------------------------------------------

relative path = Permafrost/WestDock/pickles/kmerdf_from_metagenome_fasta_6mer_incomplete.pickle
----------------------------------------------------
----------------------------------------------------
Progress: 95.37 %, local time: 23:49
Total elapsed time is 1525.16 minutes
Saving current contig kmers to pickle...
relative path = Permafrost/WestDock/pickles/kmerdf_from_metagenome_fasta_6mer_incomplete.pickle
----------------------------------------------------
----------------------------------------------------
Progress: 96.46 %, local time: 0:9
Total elapsed time is 1545.01 minutes
Saving current contig kmers to pickle...
relative path = Permafrost/WestDock/pickles/kmerdf_from_metagenome_fasta_6mer_incomplete.pickle
----------------------------------------------------
----------------------------------------------------
Progress: 97.56 %, local time: 0:29
Total elapsed time is 1565.38 minutes
Saving current contig kmers to pickle...
relative path = Permafrost/WestDock/pickles/kmerd

# writing WD fastas of PCA

In [5]:
imp.reload(pd)
WDtsnedf3kb = pd.read_pickle('Permafrost/WestDock/pickles/WDminichipCov3kbthres6mers_all_tSNEs')
WDtsnedf5kb = pd.read_pickle('Permafrost/WestDock/pickles/WDminichipCov5kbthres6mers_all_tSNEs')
WDtsnedf7kb = pd.read_pickle('Permafrost/WestDock/pickles/WDminichipCov7kbthres6mers_all_tSNEs')
WDtsnedf10kb = pd.read_pickle('Permafrost/WestDock/pickles/WDminichipCov10kbthres6mers_all_tSNEs')

In [6]:
WDtsnedf3kb.head()

,x_160mers,y_160mers,Sequence length,GC,x_PC4,y_PC4,x_PC6,y_PC6,x_PC9,y_PC9,...,x_PC26,y_PC26,x_PC38,y_PC38,x_PC55,y_PC55,x_PC78,y_PC78,x_PC113,y_PC113
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
SuperContig_NODE_1352_length_18628_cov_1.96179,31.575850,3.469471,18628,0.33,-18.332090,-20.095514,-3.475165,-29.315403,-27.643469,-3.993829,...,-29.258291,-22.604109,6.217392,53.820469,-26.478449,-18.334024,38.173321,-39.896446,21.021051,-39.888367
SuperContig_NODE_8940_length_7683_cov_0.132911,-43.299213,19.329697,7683,0.41,-11.451790,68.012817,-2.638265,55.267048,28.569019,46.891644,...,17.016922,35.047127,-20.102636,30.415081,1.964772,27.980080,17.766685,27.913815,-44.263035,45.189808
SuperContig_NODE_15749_length_5217_cov_0.446463,28.099161,16.418409,5217,0.52,90.308121,3.982028,84.414833,24.387733,81.602951,16.899786,...,78.420700,36.667538,76.324562,41.743462,68.595009,49.237881,62.431118,50.118202,18.289827,-27.063845
SuperContig_NODE_8751_length_7787_cov_54.6094,18.064379,0.208569,7787,0.41,-36.446644,-11.303999,-15.265374,-18.893314,-14.107127,5.644021,...,2.124321,-5.573134,4.644785,1.810426,-0.492613,-28.351114,5.010416,-18.267050,25.398792,24.211378
SuperContig_NODE_2002_length_15947_cov_0.98025,2.331565,2.761369,15947,0.47,36.319847,1.037394,45.580162,-53.018169,15.057109,-38.784721,...,66.262909,-22.474421,13.154781,20.223690,21.144352,-10.222426,29.156239,18.623459,17.449423,-18.079554


In [ ]:
imp.reload(minimeta)
"""
minCS ['min_cluster_size']
minS ['min_samples']
CSM ['cluster_selection_method']
ASC ['allow_single_cluster']
perp ['perplexity_na'] # this is a string of the form '40_a' or '60_n' (so perplexity and absolute or normalized tsne)
expt_name ['expt_name']
write_fasta ['write_fasta'] # save the fasta Y/n (as in: do dry run clustering first, as fasta writing takes time)
writedf ['write_df']
series ['is_this_a_series']
"""
minCS = 40
keys = ['min_cluster_size','min_samples','cluster_selection_method','allow_single_cluster'
             ,'perplexity_na','expt_name','write_fasta','write_df','is_this_a_series','outdir']
values = [minCS,1,'leaf',False,'PC13','FranklinBluffs_miniPC13','YES','YES','NO','Permafrost/FranklinBluffs/']
kwargs = dict(zip(keys,values))
for pcs in [9,13,18,26,38,55,78,113]:
    pc = str(pcs)
    kwargs['perplexity_na'] = 'PC'+pc
    kwargs['expt_name'] = 'FranklinBluffs_miniPC'+pc+'_5kbthres_minCS'+str(minCS)
    maindf_mini = minimeta.cluster_main_tsne(maindf_mini5,**kwargs) # cluster whatever pcs you want here.
minCS = 30
keys = ['min_cluster_size','min_samples','cluster_selection_method','allow_single_cluster'
             ,'perplexity_na','expt_name','write_fasta','write_df','is_this_a_series','outdir']
values = [minCS,1,'leaf',False,'PC13','FranklinBluffs_miniPC13','YES','YES','NO','Permafrost/FranklinBluffs/']
kwargs = dict(zip(keys,values))
for pcs in [9,13,18,26,38,55,78,113]:
    pc = str(pcs)
    kwargs['perplexity_na'] = 'PC'+pc
    kwargs['expt_name'] = 'FranklinBluffs_miniPC'+pc+'_5kbthres_minCS'+str(minCS)
    maindf_mini = minimeta.cluster_main_tsne(maindf_mini5,**kwargs) # cluster whatever pcs you want here.

minCS = 20
keys = ['min_cluster_size','min_samples','cluster_selection_method','allow_single_cluster'
             ,'perplexity_na','expt_name','write_fasta','write_df','is_this_a_series','outdir']
values = [minCS,1,'leaf',False,'PC13','FranklinBluffs_miniPC13','YES','YES','NO','Permafrost/FranklinBluffs/']
kwargs = dict(zip(keys,values))
for pcs in [9,13,18,26,38,55,78,113]:
    pc = str(pcs)
    kwargs['perplexity_na'] = 'PC'+pc
    kwargs['expt_name'] = 'FranklinBluffs_miniPC'+pc+'_10kbthres_minCS'+str(minCS)
    maindf_mini = minimeta.cluster_main_tsne(maindf_mini10,**kwargs) # cluster whatever pcs you want here.

minCS = 30
keys = ['min_cluster_size','min_samples','cluster_selection_method','allow_single_cluster'
             ,'perplexity_na','expt_name','write_fasta','write_df','is_this_a_series','outdir']
values = [minCS,1,'leaf',False,'PC13','FranklinBluffs_miniPC13','YES','YES','NO','Permafrost/FranklinBluffs/']
kwargs = dict(zip(keys,values))
for pcs in [9,13,18,26,38,55,78,113]:
    pc = str(pcs)
    kwargs['perplexity_na'] = 'PC'+pc
    kwargs['expt_name'] = 'FranklinBluffs_miniPC'+pc+'_10kbthres_minCS'+str(minCS)
    maindf_mini = minimeta.cluster_main_tsne(maindf_mini10,**kwargs) # cluster whatever pcs you want here.






